In [1]:
#I need to quantify diversity now

In [16]:
import glob
import os
import pandas as pd
import json
import numpy as np

# Things that should be packages

In [3]:
# from collections.abc import Iterable, MutableSequence
# import os
# import numpy as np

# class SimplePDBParser:
#     def __init__(self,data_input=None):
#         self.records = []

#         #must determine the nature of the data_input
#         if type(data_input) == None:
#             #nothing to do...
#             pass
#         elif type(data_input) == Iterable:
#             self.parse_stream(data_input)
#         elif type(data_input) == str:
#             #this could be a number of things
#             if os.path.isfile(data_input):
#                 self.parse_file(data_input)
#             else:
#                 self.parse_string(data_input)


#     #returns the number of ATOM records with unique chainID and resSeq
#     def __len__(self) -> int:
#         unique_reses = set()
#         for record in self.records:
#             if record.record_type == "ATOM":
#                 unique_reses.add(record.chainID+str(record.resSeq))
#         return len(unique_reses)

#     #collects all xyz coordinates of all ATOM records and returns them in an array in the order they appear in the pdbs
#     def coords(self) -> np.array:
#         data = []
#         for record in self.records:
#             if record.record_type == "ATOM":
#                 data.append((record.x,record.y,record.z))
#         return np.array(data)

#     def __str__(self):
#         return "\n".join([str(record) for record in self.records])

#     #returns an npose based on this pdb file
#     def npose(self):
#         raise NotImplementedError()


#     def extract_chain(self,chainID,preserve_other_records=True):
#         new_pdb = SimplePDBParser()
#         new_records = []
#         for record in self.records:
#             if preserve_other_records:
#                 if record.record_type != "ATOM" or record.chainID == chainID:
#                     new_records.append(record)

#         new_pdb.records = new_records

#         return new_pdb

#     class BasicRecord:
#         def __init__(self,line):
#             self.record_type = "BASIC"
#             self.data = line.strip()

#         def __str__(self):
#             return self.data

#         def __repr__self():
#             return str(self)

#     class AtomRecord(BasicRecord):
#         def __init__(self,line):
#             self.record_type = line[0:6].strip()
#             self.serial = int(line[6:11])
#             self.name = line[12:16].strip()
#             self.altLoc = line[16]
#             self.resName = line[17:20].strip()
#             self.chainID = line[21]
#             self.resSeq = int(line[22:26])
#             self.iCode = line[26]
#             self.x = float(line[30:38])
#             self.y = float(line[38:46])
#             self.z = float(line[46:54])
#             self.occupancy = float(line[54:60])
#             self.tempFactor = float(line[60:66])
#             self.element = line[76:78].strip()
#             self.charge = line[78:80].strip()

#         def __str__(self):
#             line = ""

#             line += self.record_type.ljust(6,' ')
#             line += str(self.serial).rjust(5,' ')
#             line += " "
#             line += self.name.ljust(4,' ')
#             line += self.altLoc
#             line += self.resName.rjust(3,' ')
#             line += " "
#             line += self.chainID
#             line += str(self.resSeq).rjust(4,' ')
#             line += self.iCode
#             line += "   "
#             line += f"{self.x:8.3f}"
#             line += f"{self.y:8.3f}"
#             line += f"{self.z:8.3f}"
#             line += f"{self.occupancy:6.2f}"
#             line += f"{self.tempFactor:6.2f}"
#             line += "          "
#             line += self.element.rjust(2,' ')
#             line += self.charge.rjust(2,' ')

#             return line


#         def __repr__self():
#             return str(self)

#     def parse_record(self,line):
#         record_type = line[0:6].strip()

#         if record_type == "ATOM":
#             return self.AtomRecord(line)
#         else:
#             return self.BasicRecord(line)

#     def parse_stream(self,pdb_stream):
#         #clear the main datatype before refilling to prevent accidental appending
#         self.records = []

#         for line in pdb_stream:
#             self.records.append(self.parse_record(line))

#     def parse_file(self,pdb_filename):
#         with open(pdb_filename,'r') as f:
#             self.parse_stream(f)

#     def parse_string(self,pdb_str:str):
#         self.parse_stream(pdb_str.split("\n"))

# The rest of it

In [4]:
with open("/home/rdkibler/projects/tiara_gen2/rotors/03_RC4_20_small_perturb/05_assemble/hbnet_mapping.json") as f:
        hbnet_mapping = json.load(f)

df = pd.read_csv("/home/rdkibler/projects/tiara_gen2/rotors/03_RC4_20_small_perturb/06_more_filtering/filtered.csv.gz")

def get_hbnet_lines(name):
    decoy = df[df['review_id'] == name]['decoy'].values[0]
    hbnet_path = hbnet_mapping[decoy]
    print(hbnet_path)
    res_i_n_map = {}
    constraints = []
    with open(hbnet_path,'r') as pdb:
        for line in pdb.readlines():
            if line.startswith("ATOM"):
                resname = line[17:20].strip()
                resi = int(line[22:26])
                res_i_n_map[resi] = resname
            if line[1:9] == "AtomPair":
                constraints.append(line[1:])
    
    bonds = []
    for constraint in constraints:
        #1st is acceptor
        #2nd is donor

        #if atom of acceptor is O, that's a bb and I might just ignore.
        acceptor_atom = constraint.split()[1]
        acceptor_resi = int(constraint.split()[2])
        donor_atom = constraint.split()[3]
        donor_resi = int(constraint.split()[4])
        if acceptor_atom != "O":
            bonds.append(((donor_resi,res_i_n_map[donor_resi]),(acceptor_resi,res_i_n_map[acceptor_resi])))
        
        
        
    #         HBNet_resis = set()

    #         #convert constraints
    #         adjusted_constraints = []
    #         len_chain_A = 66
    #         for constraint in constraints:
    #                 constraint_parts = constraint.split()
    #                 #oops! hardcoded
    #                 if int(constraint_parts[2]) <= len_chain_A:
    #                         constraint_parts[2] = f"{constraint_parts[2]}A"
    #                 else:
    #                         constraint_parts[2] = f"{int(constraint_parts[2]) + len_chain_A}B"

    #                 if int(constraint_parts[4]) <= len_chain_A:
    #                         constraint_parts[4] = f"{constraint_parts[4]}A"
    #                 else:
    #                         constraint_parts[4] = f"{int(constraint_parts[4]) + len_chain_A}B"
    #                 adjusted_constraints.append(" ".join(constraint_parts) + "\n")

    #         for constraint in adjusted_constraints:
    #                 HBNet_resis.add(constraint.split()[2])
    #                 HBNet_resis.add(constraint.split()[4])

    #         return HBNet_resis
    return bonds

In [5]:
all_passing_pdbs = glob.glob("03_passing_designs/*.pdb.gz")

In [6]:

review_ids = sorted([os.path.basename(x).split(".pdb.gz")[0] for x in all_passing_pdbs])


In [7]:

nets = {}
for ris in review_ids:
    nets[ris] = get_hbnet_lines(ris)
    


/net/scratch/rdkibler/201224_rotor_hbnet/2/RC4_20_mini_nm_00077_1.6_31_0001_0034/RC4_20_mini_nm_00077_1.6_31_0001_0034_hb.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/2/RC4_20_mini_nm_00146_1.6_31_0001_0042/RC4_20_mini_nm_00146_1.6_31_0001_0042_hb.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/2/RC4_20_mini_nm_00083_1.6_31_0001_0051/RC4_20_mini_nm_00083_1.6_31_0001_0051_hb.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/3/RC4_20_mini_nm_00137_1.6_31_0001_0057/RC4_20_mini_nm_00137_1.6_31_0001_0057_hb_0010.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/0/RC4_20_mini_nm_00152_1.6_31_0001_0045/RC4_20_mini_nm_00152_1.6_31_0001_0045_hb_0003.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/2/RC4_20_mini_nm_00074_1.6_31_0001_0012/RC4_20_mini_nm_00074_1.6_31_0001_0012_hb_0002.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/0/RC4_20_mini_nm_00184_1.6_31_0001_0044/RC4_20_mini_nm_00184_1.6_31_0001_0044_hb.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/0/RC4_20_mini_nm_00099_1.6_31_0001_0058/RC4_20_mini_nm_00099_1

In [8]:
hbnet_resises = {}
for des_id, bonds in nets.items():
    resis = set()
    for bond in bonds:
        resis.add(bond[0])
        resis.add(bond[1])
    hbnet_resises[des_id] = resis

In [9]:
hbnet_resises

{'RC_4_20_nmp1_redesign_01': {(24, 'SER'),
  (25, 'GLN'),
  (29, 'GLU'),
  (129, 'GLN'),
  (148, 'SER')},
 'RC_4_20_nmp1_redesign_02': {(21, 'HIS'), (121, 'SER'), (125, 'THR')},
 'RC_4_20_nmp1_redesign_03': {(21, 'SER'), (125, 'SER'), (151, 'SER')},
 'RC_4_20_nmp1_redesign_04': {(21, 'ASN'),
  (22, 'GLU'),
  (125, 'SER'),
  (151, 'SER'),
  (152, 'THR'),
  (164, 'SER')},
 'RC_4_20_nmp1_redesign_05': {(21, 'HIS'), (121, 'SER'), (125, 'THR')},
 'RC_4_20_nmp1_redesign_06': {(21, 'GLN'), (125, 'THR'), (160, 'ASN')},
 'RC_4_20_nmp1_redesign_07': {(21, 'GLN'), (125, 'THR'), (160, 'ASN')},
 'RC_4_20_nmp1_redesign_08': {(21, 'HIS'),
  (121, 'SER'),
  (125, 'THR'),
  (160, 'SER')},
 'RC_4_20_nmp1_redesign_09': {(21, 'GLN'), (125, 'SER'), (160, 'ASN')},
 'RC_4_20_nmp1_redesign_10': {(21, 'GLN'), (125, 'THR'), (160, 'ASN')},
 'RC_4_20_nmp1_redesign_11': {(17, 'SER'),
  (20, 'GLN'),
  (114, 'GLN'),
  (118, 'SER')},
 'RC_4_20_nmp1_redesign_12': {(21, 'GLN'), (125, 'SER'), (160, 'ASN')},
 'RC_4_20_nm

In [10]:
#I guess let's try to construct a matrix?

In [11]:

def num_shared_resis(net_resis_1,net_resis_2):
    #gonna treat SER and THR the exact same for this
    #may also collapse GLN and ASN, but let's see
    
    #cast all THR to SER
    net_resis_1 = set([(x[0],x[1].replace('THR','SER')) for x in net_resis_1])
    net_resis_2 = set([(x[0],x[1].replace('THR','SER')) for x in net_resis_2])
    print(net_resis_1)

In [12]:
#give me nets
def across_interface(resi_1, resi_2):
    chainbreak = 66
    return (resi_1 <= chainbreak) ^ (resi_2 <= chainbreak)

def pct_shared_interface_bonds(net1,net2):
#     print("net1")
#     print(net1)
#     print("net2")
#     print(net2)
    
    #cast all THR to SER
    temp = []
    for bond in net1:
        donor = list(bond[0])
        acceptor = list(bond[1])
        
        donor[1] = donor[1].replace('THR','SER')
        acceptor[1] = acceptor[1].replace('THR','SER')
        
        bond = (tuple(donor),tuple(acceptor))
        temp.append(bond)
    net1 = temp
    
    temp = []
    for bond in net2:
        donor = list(bond[0])
        acceptor = list(bond[1])
        
        donor[1] = donor[1].replace('THR','SER')
        acceptor[1] = acceptor[1].replace('THR','SER')
        
        bond = (tuple(donor),tuple(acceptor))
        temp.append(bond)
    net2 = temp
    
#     print("convert")
#     print("net1")
#     print(net1)
#     print("net2")
#     print(net2)
#     print("interface_bonds_1")
    interface_bonds_1 = set([(donor,acceptor) for donor,acceptor in net1 if across_interface(donor[0],acceptor[0])])
#     print(interface_bonds_1)
    num_interface_bonds_1 = len(interface_bonds_1)
#     print(num_interface_bonds_1)
#     print("interface_bonds_2")
    interface_bonds_2 = set([(donor,acceptor) for donor,acceptor in net2 if across_interface(donor[0],acceptor[0])])
#     print(interface_bonds_2)
    num_interface_bonds_2 = len(interface_bonds_2)
#     print(num_interface_bonds_2)
    
    unique_interface_bonds = interface_bonds_1.union(interface_bonds_2)
    shared_interface_bonds = interface_bonds_1.intersection(interface_bonds_2)
    
    percent_shared = len(shared_interface_bonds) / len(unique_interface_bonds)
    return percent_shared

In [13]:
#same = all identical interface bonds = 0
#different = no identical interface bonds = 1
#in between = num identical interface bonds / num unique interface bonds

In [14]:
net1 = list(nets.values())[1]
net2 = list(nets.values())[7]
pct_shared_interface_bonds(net1,net2)

1.0

In [19]:
net_pair_similarity = []


net_pair_similarity_matrix = np.zeros([len(nets),len(nets)])
names = sorted(list(nets.keys()))

for i,k1 in enumerate(names):
    for j,k2 in enumerate(names):
        net_pair_similarity_matrix[i,j] = pct_shared_interface_bonds(nets[k1],nets[k2])

In [20]:
net_pair_similarity_matrix

array([[1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 1.  , 0.25],
       [0.  , 0.  , 0.  , ..., 0.  , 0.25, 1.  ]])

In [1]:
from sklearn.cluster import DBSCAN
clusters = DBSCAN(min_samples=1).fit_predict(net_pair_similarity_matrix)
clusters

NameError: name 'don_coords' is not defined

In [22]:
len(clusters)

79

In [23]:
from collections import defaultdict

In [24]:
#sorting keys into clusters
cluster_groups = defaultdict(lambda:[])
for cluster_number,name in zip(clusters,names):
    print(name, cluster_number)
    cluster_groups[cluster_number].append(name)

RC_4_20_nmp1_redesign_01 0
RC_4_20_nmp1_redesign_02 1
RC_4_20_nmp1_redesign_03 2
RC_4_20_nmp1_redesign_04 3
RC_4_20_nmp1_redesign_05 1
RC_4_20_nmp1_redesign_06 4
RC_4_20_nmp1_redesign_07 4
RC_4_20_nmp1_redesign_08 1
RC_4_20_nmp1_redesign_09 4
RC_4_20_nmp1_redesign_10 4
RC_4_20_nmp1_redesign_11 5
RC_4_20_nmp1_redesign_12 4
RC_4_20_nmp1_redesign_13 6
RC_4_20_nmp1_redesign_14 2
RC_4_20_nmp1_redesign_15 7
RC_4_20_nmp1_redesign_16 8
RC_4_20_nmp1_redesign_17 1
RC_4_20_nmp1_redesign_18 4
RC_4_20_nmp1_redesign_19 4
RC_4_20_nmp1_redesign_20 4
RC_4_20_nmp1_redesign_21 4
RC_4_20_nmp1_redesign_22 4
RC_4_20_nmp1_redesign_23 5
RC_4_20_nmp1_redesign_24 4
RC_4_20_nmp1_redesign_25 9
RC_4_20_nmp1_redesign_26 10
RC_4_20_nmp1_redesign_27 1
RC_4_20_nmp1_redesign_28 4
RC_4_20_nmp1_redesign_29 4
RC_4_20_nmp1_redesign_30 11
RC_4_20_nmp1_redesign_31 4
RC_4_20_nmp1_redesign_32 12
RC_4_20_nmp1_redesign_33 13
RC_4_20_nmp1_redesign_34 4
RC_4_20_nmp1_redesign_35 14
RC_4_20_nmp1_redesign_36 4
RC_4_20_nmp1_redesign_3

In [28]:
pct_shared_interface_bonds(nets['RC_4_20_nmp1_redesign_37'],nets['RC_4_20_nmp1_redesign_49'])

0.3333333333333333

In [29]:
nets['RC_4_20_nmp1_redesign_37']

[((129, 'SER'), (28, 'SER')),
 ((144, 'HIS'), (28, 'SER')),
 ((128, 'SER'), (144, 'HIS')),
 ((130, 'ARG'), (129, 'SER'))]

In [30]:
nets['RC_4_20_nmp1_redesign_49']

[((28, 'SER'), (129, 'SER')),
 ((144, 'HIS'), (28, 'SER')),
 ((128, 'SER'), (144, 'HIS'))]

In [158]:
cluster_groups

defaultdict(<function __main__.<lambda>()>,
            {0: ['RC_4_20_nmp1_redesign_01'],
             1: ['RC_4_20_nmp1_redesign_02',
              'RC_4_20_nmp1_redesign_05',
              'RC_4_20_nmp1_redesign_08',
              'RC_4_20_nmp1_redesign_17',
              'RC_4_20_nmp1_redesign_27',
              'RC_4_20_nmp1_redesign_55'],
             2: ['RC_4_20_nmp1_redesign_03',
              'RC_4_20_nmp1_redesign_14',
              'RC_4_20_nmp1_redesign_64'],
             3: ['RC_4_20_nmp1_redesign_04'],
             4: ['RC_4_20_nmp1_redesign_06',
              'RC_4_20_nmp1_redesign_07',
              'RC_4_20_nmp1_redesign_09',
              'RC_4_20_nmp1_redesign_10',
              'RC_4_20_nmp1_redesign_12',
              'RC_4_20_nmp1_redesign_18',
              'RC_4_20_nmp1_redesign_19',
              'RC_4_20_nmp1_redesign_20',
              'RC_4_20_nmp1_redesign_21',
              'RC_4_20_nmp1_redesign_22',
              'RC_4_20_nmp1_redesign_24',
             

In [ ]:
#ok, there's 29 clusters (of designs with identical cross-interface bonds)
#This means there's a theoretical maximum of 29 different designs

#how does this compare to the full set pre-design?

#I'll do that later